In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 4.0 MB/s 
     |████████████████████████████████| 895 kB 38.4 MB/s 
     |████████████████████████████████| 3.3 MB 42.0 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 636 kB 37.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import re
# from textblob import TextBlob

path = "/content/gdrive/MyDrive/enter_weights/"
df = pd.read_csv(path+"clean_train.csv")
df_test = pd.read_csv(path+"clean_test.csv")

df.head()

,text,aspect,label,clean_text,clean_aspect
0,can you check whether its cancelled completely?,cancelled,1,can you check whether its canceled completely?,canceled
1,cannot rely on both milk delivery and grocery ...,Milk,0,cannot rely on both milk delivery and grocer i...,milk
2,"I get no notification, however the app is real...",notification,0,"i get no ratification, however the pp is reall...",ratification
3,"Love this app, but would love it even more if ...",view,1,"love this pp, but would love it even more if w...",view
4,it does not let me load a clip on the scene,load,0,it does not let me load a clip on the scene,load


In [ ]:
punctuation = '!"#$%()*+-/:;<=>@[\\]^_`{|}~'

df["clean_text"] = df["text"].apply(lambda x: re.sub(r"http\S+", "", x))
df["clean_text"] = df["clean_text"].apply(lambda x: "".join(ch for ch in x if ch not in set(punctuation)))
df["clean_text"] = df["clean_text"].str.lower()
df["clean_text"] = df["clean_text"].str.replace("[0-9]", " ")
df["clean_text"] = df["clean_text"].apply(lambda x:" ".join(x.split()))
# df["clean_text"] = df["clean_text"].apply(lambda x: str(TextBlob(x).correct()))


df["clean_aspect"] = df["aspect"].apply(lambda x: re.sub(r"http\S+", "", x))
df["clean_aspect"] = df["clean_aspect"].apply(lambda x: "".join(ch for ch in x if ch not in set(punctuation)))
df["clean_aspect"] = df["clean_aspect"].str.lower()
df["clean_aspect"] = df["clean_aspect"].str.replace("[0-9]", " ")
df["clean_aspect"] = df["clean_aspect"].apply(lambda x:" ".join(x.split()))
# df["clean_aspect"] = df["clean_aspect"].apply(lambda x: str(TextBlob(x).correct()))


df_test["clean_text"] = df_test["text"].apply(lambda x: re.sub(r"http\S+", "", x))
df_test["clean_text"] = df_test["clean_text"].apply(lambda x: "".join(ch for ch in x if ch not in set(punctuation)))
df_test["clean_text"] = df_test["clean_text"].str.lower()
df_test["clean_text"] = df_test["clean_text"].str.replace("[0-9]", " ")
df_test["clean_text"] = df_test["clean_text"].apply(lambda x:" ".join(x.split()))
# df_test["clean_text"] = df_test["clean_text"].apply(lambda x: str(TextBlob(x).correct()))


df_test["clean_aspect"] = df_test["aspect"].apply(lambda x: re.sub(r"http\S+", "", x))
df_test["clean_aspect"] = df_test["clean_aspect"].apply(lambda x: "".join(ch for ch in x if ch not in set(punctuation)))
df_test["clean_aspect"] = df_test["clean_aspect"].str.lower()
df_test["clean_aspect"] = df_test["clean_aspect"].str.replace("[0-9]", " ")
df_test["clean_aspect"] = df_test["clean_aspect"].apply(lambda x:" ".join(x.split()))
# df_test["clean_aspect"] = df_test["clean_aspect"].apply(lambda x: str(TextBlob(x).correct()))


# df.to_csv(path+"clean_train.csv", index=False)
# df_test.to_csv(path+"clean_test.csv", index=False)

In [ ]:
import numpy as np


def build_k_datasets(texts, phrases, targets, splits=5, random_state=0):
    """
    Parameters
    ----------
    texts: np.array[(str)] or List[str]
      A array of texts
    phrases: np.array[(str)] or List[str]
      A array of phrases
    splits: int
      Number of splits
    random_state: int
      Random state for reprodubility

    Return
    ------
    train_dataset: List[List[str]]
        A k fold train split of strings in this format (Texts_array, Phrase_array, targets_array)
    test_dataset: List[List[str]]
        A k fold test split of strings in this format (Texts_array, Phrase_array, targets_array)
    """
    from sklearn.model_selection import StratifiedKFold

    skf = StratifiedKFold(n_splits=splits, shuffle=True,
                          random_state=random_state)

    if type(texts) != np.ndarray:
        texts = np.array(texts)
    if type(phrases) != np.ndarray:
        phrases = np.array(phrases)
    if type(targets) != np.ndarray:
        targets = np.array(targets)

    skf.get_n_splits(texts, targets)
    train_dataset = []
    test_dataset = []
    for train_index, test_index in skf.split(texts, targets):
        # print("TRAIN:", train_index, "TEST:", test_index)
        text_train, text_test = texts[train_index], texts[test_index]
        phrase_train, phrase_test = phrases[train_index], phrases[test_index]
        targets_train, targets_test = targets[train_index], targets[test_index]

        train_dataset.append((text_train, phrase_train, targets_train))
        test_dataset.append((text_test, phrase_test, targets_test))

    return train_dataset, test_dataset



In [ ]:
from logging import error
import torch
import numpy as np
from transformers import BertTokenizerFast
from torch.utils.data.dataloader import default_collate
import torch.nn.functional as F
import re
import warnings
warnings.filterwarnings("ignore")

class BERT_Dataset_Type_1:
    def __init__(self,
                 texts,
                 phrases,
                 targets,
                 tokenizer_save_path,
                 tokenizer_method=BertTokenizerFast,
                 max_len=128):
        self.texts = texts
        self.phrases = phrases
        self.targets = targets
        self.tokenizer = tokenizer_method.from_pretrained(
            tokenizer_save_path,
            do_lower_case=True,
        )
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        # print(item)
        # Calling the indexes of texts and phrases
        text = self.texts[item]
        phrase = self.phrases[item]
        text = " ".join(text.split()).lower()
        phrase = " ".join(phrase.split()).lower()
        # print(text, len(text))
        # print(phrase)
        # To find entries of phrases in text
        # Example text = "I am a monkey" , phrase = "monkey"
        # char_mask = [0,0,0,0,0,0,0,1,1,1,1,1,1]
        char_mask = [(m.start(0), m.end(0))
                     for m in re.finditer(phrase.lower(), text.lower())]
        tok = self.tokenizer(text, return_offsets_mapping=True)
        # print(char_mask)
        tok_ids = tok.input_ids
        tok_offsets = tok.offset_mapping
        buffer = [0] * len(text)
        token_type_ids = [0]*(len(tok_ids)-2)
        # print("tok ids", tok_ids, len(tok_ids))
        # print("token type ids", token_type_ids, len(token_type_ids))
        # print(self.tokenizer.encode(text).tokens)
        # For each token in the phrase, we represent it with 1
        # text = "I am a monkey"
        # token_type_ids = [0, 0, 0, 1]
        for m in char_mask:
            for i in range(m[0], m[1]):
                buffer[i] = 1

        for j, (offset1, offset2) in enumerate(tok_offsets):
            if sum(buffer[offset1:offset2]) > 0 and j < len(tok_ids)-2:
                token_type_ids[j] = 1
        
        if sum(token_type_ids) is not 0:
          token_type = [0] + token_type_ids + [0]
        else:
          token_type = [1] + token_type_ids + [0]

        tok_ids = torch.tensor(tok_ids, dtype=torch.long)
        tok_masks = torch.tensor(tok.attention_mask, dtype=torch.long)
        tok_type = torch.tensor(token_type, dtype=torch.long)
        targets = torch.tensor(self.targets[item], dtype=torch.float)
        # print(tok_ids.shape, tok_masks.shape, tok_type.shape, targets)
        return tok_ids, tok_masks, tok_type, targets


def custom_collate(data):
    max_len = 0
    ids, masks, tok_types, targets = map(list, zip(*data))
    for id in ids:
        # print(id.shape[0])
        max_len = max(max_len, id.shape[0])
    
    padded_ids, padded_masks, padded_tok_types = [], [], []
    
    for id in ids:
        pad_size = max_len - id.shape[0]
        padded_ids.append(F.pad(id, (0, pad_size), "constant", 0))

    for mask in masks:
        pad_size = max_len - mask.shape[0]
        padded_masks.append(F.pad(mask, (0, pad_size), "constant", 0))

    for tok_type in tok_types:
        pad_size = max_len - tok_type.shape[0]
        padded_tok_types.append(F.pad(tok_type, (0, pad_size), "constant", 0))

    padded_ids = torch.stack(padded_ids, dim=0)
    padded_masks = torch.stack(padded_masks, dim=0)
    padded_tok_types = torch.stack(padded_tok_types, dim=0)
    targets = torch.stack(targets, dim=0)
    return padded_ids, padded_masks, padded_tok_types, targets



In [ ]:
import torch.nn as nn
import transformers
from transformers import AutoModel
class Embedding_model(nn.Module):
    def __init__(self, BERT_PATH, base_model_dim):
        super(Embedding_model, self).__init__()
        self.bert = AutoModel.from_pretrained(BERT_PATH)
        self.bert_drop = nn.Dropout(0.3)
        self.head1 = nn.Linear(base_model_dim, 256)
        self.head2 = nn.Linear(256, 3)
        self.dropout = nn.Dropout(0.1)

    def forward(self, ids, attn_mask, token_types):
        bert_output = self.bert(ids, attn_mask)
        embed_extract = []
        for bert_embed, token_type in zip(bert_output[0], token_types):
            embed_extract.append(torch.mean(bert_embed[token_type == 1], axis=0))

        embed_extract = torch.stack(embed_extract, dim=0)
        output = self.head1(embed_extract)
        output = F.leaky_relu(self.dropout(output))
        output = self.head2(output)
        return output


In [ ]:

import time
import numpy as np
import torch
import torch.nn as nn
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import pandas as pd


def classify_default(model, tok_ids, attn_mask, tok_type, device):

    tok_ids = tok_ids.to(device)
    attn_mask = attn_mask.to(device)
    tok_type = tok_type.to(device)
    return model(tok_ids,
                 attn_mask,
                 tok_type)


def loss_fn_cross_entropy(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets.to(int))


class Trainer:
    """
    Will be used for Trainer Class for training the model

    1. Will train the model
    2. Will evaluate the model
    3. Will save the torch model
    4. Will save the model to onnx
    5. Can connect it to wandb
    """

    def __init__(self,
                 model,
                 classify,
                 config,
                 device=None
                 ):
        if device is not None:
            self.device = device
        else:
            self.device = torch.device(
                "cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.model = model.to(self.device)
        self.optimizer = AdamW(model.parameters(),
                               lr=config["learning_rate"])
        self.criterion = loss_fn_cross_entropy
        self.classify = classify
        self.config = config
        num_train_steps = int(config["df_len"] / config["train_batch_size"] * config["num_epochs"])
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps=0,
            num_training_steps=num_train_steps
        )

    def train_classification(self,
                             train_dataloader,
                             val_dataloader,
                             start_step=0):

        training_log = []

        for epoch in range(self.config["num_epochs"]):
            start_time = time.time()
            self.model.train()
            running_loss = []

            for batch_idx, batch in tqdm(enumerate(train_dataloader),
                                         total=len(train_dataloader),
                                         leave=False):
                self.optimizer.zero_grad()
                tok_ids, attn_mask, tok_type, targets = batch

                predictions = classify(self.model, tok_ids, attn_mask, tok_type, self.device)
                loss = self.criterion(predictions, targets.to(self.device))
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                running_loss.append(loss.item())

                start_step += 1

            running_loss = np.mean(running_loss)
            overall_acc, acc_1, acc_2, acc_3, f1, val_loss = self.get_validation(
                val_dataloader)
            elapsed = time.time() - start_time

            training_log.append([epoch, running_loss, val_loss,
                                 acc_1, acc_2, acc_3, f1])
            print(
                "| epoch {:3d} | train loss {:8.5f} | Val loss {:8.5f} | Overall acc {:5.3f} | "
                "class_1_acc {:5.3f} | class_2_acc {:5.3f} | class_3_acc {:5.3f} | "
                "f1_score {:5.3f} | Time Taken {:5.3f}s |".format(
                    epoch, running_loss, val_loss, overall_acc,
                    acc_1, acc_2, acc_3, f1, elapsed
                )
            )
            print(" ".join("-" for _ in range(75)))

        return training_log

    def get_validation(self, val_dataloader):
        self.model.eval()
        running_loss = []
        y_pred = []
        y_true = []
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_dataloader):
                tok_ids, attn_mask, tok_type, targets = batch

                predictions = classify(self.model, tok_ids, attn_mask, tok_type, self.device)
                loss = self.criterion(
                    predictions, targets.to(self.device))
                predictions = torch.nn.functional.softmax(
                    predictions, 1).detach().cpu()
                predictions = torch.argmax(predictions, dim = 1)
                running_loss.append(loss.item())
                y_pred += predictions.tolist()
                y_true += targets.cpu().tolist()

        f1 = f1_score( y_true, y_pred, average="weighted", zero_division=1)
        overall_acc = np.mean(np.array(y_true) == np.array(y_pred))
        matrix = confusion_matrix(y_true, y_pred)
        matrix = matrix.diagonal()/matrix.sum(axis=1)
        return overall_acc, matrix[0], matrix[1], matrix[2], f1, np.mean(running_loss)


# BERT study using Embedding vectors

In [ ]:
from transformers import AutoTokenizer

config = {
    "splits": 5,
    "seed": 0,
    "train_batch_size": 32,
    "val_batch_size": 64,
    "num_workers": 2,
    "learning_rate": 5e-5,
    "df_len": 3200,
    "num_epochs": 7,
    "max_len": 128,
    "checkpoint": "huawei-noah/TinyBERT_General_4L_312D",
    "classify": classify_default,
    "base_model_dim": 312

}
texts_all = df["clean_text"].to_numpy()
phrases_all = df["clean_aspect"].to_numpy()
targets_all = df["label"].to_numpy()

train_dataset, test_dataset = build_k_datasets(texts_all, 
                                               phrases_all, 
                                               targets_all, 
                                               config["splits"], 
                                               config["seed"])

for i, dataset in enumerate(zip(train_dataset, test_dataset)):
    print(f"Training for fold {i}")
    print(" ".join("=" for _ in range(75)))
    checkpoint = config["checkpoint"]
    train, test = dataset
    model = Embedding_model(checkpoint, config["base_model_dim"])
    texts, phrases, targets = train
    val_texts, val_phrases, val_targets = test

    train_dataset = BERT_Dataset_Type_1(texts, phrases, targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config["train_batch_size"], 
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )

    val_dataset = BERT_Dataset_Type_1(val_texts, val_phrases, val_targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    val_data_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=config["val_batch_size"],
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )
    engine = Trainer(model, config["classify"], config)

    log = engine.train_classification(
        train_data_loader,
        val_data_loader
    )


Training for fold 0
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =


Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'fit_denses.0.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'fit_denses.3.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'fit_denses.4.bias', 'cls.seq_relationship.weight', 'fit_denses.2.bias', 'fit_denses.1.weight', 'fit_denses.4.weight', 'fit_denses.1.bias', 'fit_denses.2.weight', 'fit_denses.0.bias', 'cls.seq_relationship.bias', 'fit_denses.3.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing

| epoch   0 | train loss  0.88074 | Val loss  0.78200 | Overall acc 0.652 | class_1_acc 0.792 | class_2_acc 0.610 | class_3_acc 0.478 | f1_score 0.647 | Time Taken 10.376s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   1 | train loss  0.69127 | Val loss  0.72471 | Overall acc 0.690 | class_1_acc 0.815 | class_2_acc 0.587 | class_3_acc 0.615 | f1_score 0.686 | Time Taken 10.246s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   2 | train loss  0.59751 | Val loss  0.69457 | Overall acc 0.708 | class_1_acc 0.708 | class_2_acc 0.718 | class_3_acc 0.693 | f1_score 0.708 | Time Taken 10.305s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


...

| epoch   2 | train loss  0.58317 | Val loss  0.73273 | Overall acc 0.711 | class_1_acc 0.714 | class_2_acc 0.740 | class_3_acc 0.670 | f1_score 0.712 | Time Taken 10.765s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   3 | train loss  0.53494 | Val loss  0.74912 | Overall acc 0.698 | class_1_acc 0.690 | class_2_acc 0.702 | class_3_acc 0.704 | f1_score 0.699 | Time Taken 10.822s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   4 | train loss  0.47132 | Val loss  0.74444 | Overall acc 0.710 | class_1_acc 0.693 | class_2_acc 0.783 | class_3_acc 0.646 | f1_score 0.711 | Time Taken 10.755s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   5 | train loss  0.41014 | Val loss  0.76037 | Overall acc 0.706 | class_1_acc 0.673 | class_2_acc 0.795 | class_3_acc 0.650 | f1_score 0.707 | Time Taken 10.702s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   6 | train loss  0.37692 | Val loss  0.74892 | Overall acc 0.716 | class_1_acc 0.750 | class_2_acc 0.733 | class_3_acc 0.641 | f1_score 0.716 | Time Taken 10.851s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


# Tiny BERT study using Embedding vectors

In [ ]:
from transformers import AutoTokenizer

config = {
    "splits": 5,
    "seed": 0,
    "train_batch_size": 8,
    "val_batch_size": 16,
    "num_workers": 2,
    "learning_rate": 5e-6,
    "df_len": 3200,
    "num_epochs": 5,
    "max_len": 128,
    "checkpoint": "bert-base-uncased",
    "classify": classify_default,
    "base_model_dim": 768
}
texts_all = df["clean_text"].to_numpy()
phrases_all = df["clean_aspect"].to_numpy()
targets_all = df["label"].to_numpy()

train_dataset, test_dataset = build_k_datasets(texts_all, 
                                               phrases_all, 
                                               targets_all, 
                                               config["splits"], 
                                               config["seed"])

for i, dataset in enumerate(zip(train_dataset, test_dataset)):
    print(f"Training for fold {i}")
    print(" ".join("=" for _ in range(75)))
    checkpoint = config["checkpoint"]
    train, test = dataset
    model = Embedding_model(checkpoint, config["base_model_dim"])
    texts, phrases, targets = train
    val_texts, val_phrases, val_targets = test

    train_dataset = BERT_Dataset_Type_1(texts, phrases, targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config["train_batch_size"], 
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )

    val_dataset = BERT_Dataset_Type_1(val_texts, val_phrases, val_targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    val_data_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=config["val_batch_size"],
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )
    engine = Trainer(model, config["classify"], config)

    log = engine.train_classification(
        train_data_loader,
        val_data_loader
    )


Training for fold 0
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


| epoch   0 | train loss  0.97646 | Val loss  0.78559 | Overall acc 0.645 | class_1_acc 0.711 | class_2_acc 0.602 | class_3_acc 0.590 | f1_score 0.644 | Time Taken 88.483s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   1 | train loss  0.69468 | Val loss  0.71955 | Overall acc 0.680 | class_1_acc 0.735 | class_2_acc 0.664 | class_3_acc 0.610 | f1_score 0.680 | Time Taken 88.441s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   2 | train loss  0.57811 | Val loss  0.70006 | Overall acc 0.699 | class_1_acc 0.735 | class_2_acc 0.683 | class_3_acc 0.659 | f1_score 0.699 | Time Taken 88.249s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


...

| epoch   0 | train loss  0.98944 | Val loss  0.77851 | Overall acc 0.705 | class_1_acc 0.762 | class_2_acc 0.760 | class_3_acc 0.544 | f1_score 0.703 | Time Taken 88.566s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   1 | train loss  0.71681 | Val loss  0.66452 | Overall acc 0.728 | class_1_acc 0.798 | class_2_acc 0.760 | class_3_acc 0.573 | f1_score 0.726 | Time Taken 88.494s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   2 | train loss  0.58061 | Val loss  0.63379 | Overall acc 0.743 | class_1_acc 0.783 | class_2_acc 0.740 | class_3_acc 0.680 | f1_score 0.743 | Time Taken 88.455s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   3 | train loss  0.49271 | Val loss  0.64159 | Overall acc 0.733 | class_1_acc 0.750 | class_2_acc 0.725 | class_3_acc 0.714 | f1_score 0.734 | Time Taken 88.484s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   4 | train loss  0.44635 | Val loss  0.64572 | Overall acc 0.739 | class_1_acc 0.804 | class_2_acc 0.709 | class_3_acc 0.670 | f1_score 0.738 | Time Taken 88.370s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
